### `Dependecies`

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import types as T

spark = SparkSession.builder.appName('abc')\
.config('spark.driver.memory' , '64g')\
.config('spark.sql.legacy.timeParserPolicy' , 'LEGACY')\
.config("spark.sql.shuffle.partitions" , "100")\
.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/13 14:04:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/13 14:04:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### `Config`

In [2]:
raw_data_path = '/home/data/manu/sample/cibil/'
prep_data_path = '/home/jupyter-monica.marmit/feature_enginerring/cibil/output/'

### `3K Feature Creation`

In [3]:
mon = [3, 6, 12, 24]
base_prod = ['GL','CC','TW','AL','HL']
oth_list = ["PLlt5","PLgt5","PLlt1L","PLgt1L","NAGRI","SEC","UNSEC"]
categories = ['Overall','PLlt5','PLgt5','PLlt1L','PLgt1L','GL','CC','TW','AL','HL','SEC','UNSEC','NAGRI']
categories2 = ['Overall','PLlt10k','PLlt1L','PLgt1L','GL','CC','TW','AL','HL','SEC','UNSEC','NAGRI']

In [4]:
expr_3k1 =["*"]\
+ ["case  when ACCOUNT_TYPE = 'Personal Loan' then 'PL' \
        when ACCOUNT_TYPE = 'Gold Loan'  then 'GL' \
        when ACCOUNT_TYPE = 'Credit Card' then 'CC' \
        when ACCOUNT_TYPE = 'Two-wheeler Loan' then 'TW' \
        when ACCOUNT_TYPE = 'Auto Loan (Personal)'  then 'AL' \
        when ACCOUNT_TYPE = 'Housing Loan'  then 'HL' \
        else 'others' end as prod_3k"] \
+ ["case  when ACCOUNT_TYPE = 'Personal Loan' and HIGHEST_CREDIT_OR_LOAN_AMOUNT <=5000 then 1 else null end as PLlt5_flag"]\
+ ["case  when ACCOUNT_TYPE = 'Personal Loan' and HIGHEST_CREDIT_OR_LOAN_AMOUNT > 5000 then 1 else null end as PLgt5_flag"]\
+ ["case  when ACCOUNT_TYPE = 'Personal Loan' and HIGHEST_CREDIT_OR_LOAN_AMOUNT <=10000 then 1 else null end as PLlt10k_flag"]\
+ ["case  when ACCOUNT_TYPE = 'Personal Loan' and HIGHEST_CREDIT_OR_LOAN_AMOUNT <=100000 then 1 else null end as PLlt1L_flag"]\
+ ["case  when ACCOUNT_TYPE = 'Personal Loan' and HIGHEST_CREDIT_OR_LOAN_AMOUNT > 100000 then 1 else null end as PLgt1L_flag"]\
+ ["case when ACCOUNT_TYPE in ('Other','Auto Loan (Personal)','Housing Loan','Property Loan','Loan Against Shares/Securities','Personal Loan','Consumer Loan','Gold Loan','Education Loan','Loan to Professional','Credit Card','Overdraft','Two-wheeler Loan','Loan Against Bank Deposits','Commercial Vehicle Loan','Secured Credit Card','Used Car Loan','Construction Equipment Loan','Corporate Credit Card','Kisan Credit Card','Business Loan - General','Business Loan - Priority Sector - Small Business','Business Loan - Priority Sector - Others','Business Non - Funded Credit Facility - General','Business Non - Funded Credit Facility - Priority Sector - Small Business','Business Loan Against Bank Deposits','Microfinance Business Loan','Microfinance Personal Loan','Microfinance Housing Loan','Microfinance Others','Business Loan - Unsecured') \
            then 1 else null end as NAGRI_flag"]\
+ ["case when ACCOUNT_TYPE in ('Auto Loan (Personal)','Property Loan','Housing Loan','Gold Loan','Two-wheeler Loan','Commercial Vehicle Loan','Used Car Loan','Construction Equipment Loan','Microfinance Housing Loan') then 1 else null end as SEC_flag"]\
+ ["case when ACCOUNT_TYPE in ('Personal Loan','Consumer Loan','Credit Card','Kisan Credit Card','Loan to Professional','Microfinance Personal Loan') then 1 else null end as UNSEC_flag"]\
+ ["1 as Overall_flag"] \
+ ["case when ACCOUNT_TYPE = '{a}' then 1 else null end as {b}_flag".format(a= i, b= j) for i,j in [('Gold Loan','GL'), ('Credit Card', 'CC'), ('Two-wheeler Loan', 'TW'), ('Auto Loan (Personal)', 'AL'), ('Housing Loan', 'HL')]] \
+ ["months_between(trunc(run_dt, 'month'), trunc(dateopened, 'month')) as mn_d_ret_dt_fl_3k"]\
+ ["case when dateclosed < run_dt then 1 else 0 end as closed_date_new"]\
+ ["case when ((closed_date_new = 1) or (is_written_off_or_settled = 1) or (balance_3k = 0 and ACCOUNT_TYPE not in ('Credit Card','Overdraft',14,16,'Secured Credit Card','Corporate Credit Card','Kisan Credit Card','Business Non - Funded Credit Facility - General','Business Non - Funded Credit Facility - Priority Sector - Small Business',57,58))) \
    then 1 else 0 end as is_closed"] \
+ ["case when is_closed = 0 then 1 else null end as is_open"] \
+ ["case when months_between(trunc(run_dt, 'month'), trunc(datereported, 'month')) <={a} \
    then 1 else null end as {a}mn_diff_retro_report_dt_flag".format(a = i) for i in [3,6,12,24]] \
+ ["case when is_open * {cat}_flag = 1 then balance_3k else null end as {cat}_balance".format(cat = i) for i in categories] \
+ ["case when is_open * {cat}_flag = 1 then HIGHEST_CREDIT_OR_LOAN_AMOUNT else null end as {cat}_HIGHEST_CREDIT_OR_LOAN_AMOUNT".format(cat = i) for i in categories] \
+ ["case \
    when {cat}_flag * 3mn_diff_retro_report_dt_flag *is_open = 1 then dateopened else null \
    end as dateopened_{cat}".format(cat = i) for i in categories]

In [5]:
expr_3k2 = ["*"]\
+[
    f"""
    row_number() 
    over(
        partition by MemberReference, 3mn_diff_retro_report_dt_flag, is_open 
        order by (case when dateopened_{cat} is null then 1 else 0 end), dateopened_{cat}, HIGHEST_CREDIT_OR_LOAN_AMOUNT desc
    )
    as row_num_first_{cat}""" for cat in categories] \
+[
    f"""
    row_number() 
    over(
        partition by MemberReference, 3mn_diff_retro_report_dt_flag, is_open 
        order by (case when dateopened_{cat} is null then 1 else 0 end), dateopened_{cat} desc, HIGHEST_CREDIT_OR_LOAN_AMOUNT desc
    )
    as row_num_latest_{cat}""" for cat in categories] \
+ ["case when prod_3k = '{prd}' and mn_d_ret_dt_fl_3k <={mn} then 1 else null end nbr_of_open_accnts_{prd}_{mn}M".format(prd=prd,mn=mn) \
  for prd in base_prod for mn in mon]\
+ ["case when mn_d_ret_dt_fl_3k <={mn} then 1 else null end as nbr_of_open_accnts_Overall_{mn}M".format(mn=mn) for mn in mon]\
+ ["case when {prd}_flag=1 and mn_d_ret_dt_fl_3k <={mn} then 1 else null end as nbr_of_open_accnts_{prd}_{mn}M".format(prd=prd,mn=mn) \
   for prd in oth_list for mn in mon]\
+ ["case when prod_3k = '{prd}' and mn_diff_retro_report <={mn} and is_closed=0 then 1 else null end as nbr_of_live_accnts_rep{prd}_{mn}M"\
   .format(prd=prd,mn=mn) for prd in base_prod for mn in mon]\
+ ["case when mn_diff_retro_report <={mn} and is_closed=0 then 1 else null end as nbr_of_live_accnts_repOverall_{mn}M".format(mn=mn) for mn in mon]\
+ ["case when {prd}_flag=1 and mn_diff_retro_report <={mn} and is_closed=0 then 1 else null end as nbr_of_live_accnts_rep{prd}_{mn}M".format(prd=prd,mn=mn) \
   for prd in oth_list for mn in mon]\
+ ["case when {cat}_flag = 1 and mn_diff_retro_report <={mn} and is_closed=0 and balance_3k is null then 0 \
    when {cat}_flag = 1 and mn_diff_retro_report <={mn} and is_closed=0 and balance_3k is not null then balance_3k else null end as tot_live_bal_{cat}_{mn}M"\
   .format(cat=cat,mn=mn) for cat in categories for mn in mon]\
+ [
    """
    case 
    when {cat}_flag = 1 and mn_diff_retro_report <={mn} and is_closed=0 and HIGHEST_CREDIT_OR_LOAN_AMOUNT is null then 0 \
    when {cat}_flag = 1 and mn_diff_retro_report <={mn} and is_closed=0 and HIGHEST_CREDIT_OR_LOAN_AMOUNT is not null then HIGHEST_CREDIT_OR_LOAN_AMOUNT \
    else null end as tot_live_sanction_{cat}_{mn}M""".format(cat=cat,mn=mn) for cat in categories for mn in mon
]\
+ ["case when balance_3k/HIGHEST_CREDIT_OR_LOAN_AMOUNT is null then 0 \
       else round(balance_3k/HIGHEST_CREDIT_OR_LOAN_AMOUNT,3) \
   end as mon_util"] \
+ ["months_between(trunc(dateclosed, 'month'), trunc(dateopened, 'month')) as month_diff_open_close"] \
+ ["months_between(trunc(run_dt, 'month'), trunc(payment_history_start_date, 'month')) as month_lag"] \
+ ["case when month_lag <= {mn} then 1 else null end as month_lag_{mn}M".format(mn=mn) for mn in [1,3,6,12,24]] \
+ [f"case when " + " or ".join([f"dpd_bucket_{i} >= 0 " for i in range(1, mn+1)]) + f"then 1 else null end as dpd_present_{mn}M_flag" for mn in [1,3,6,12,24]]

In [6]:
expr_3k3 = ["*"]
 
for mon in [1, 3, 6, 12, 24]:
    for bkt, dpd in [(1, '0'), (2, '30'), (3, '60'), (4, '90'), (6, '180')]:
        expr_3k3 += [" + ".join([f"int(dpd_bucket_{i} >= {bkt})" for i in range(1,mon+1)])+f" as dpd_gt_{dpd}_{mon}m_count"]
        
expr_3k3 += ["case when dpd_gt_{dpd}_{mon}m_count >= 1 then 1 else null end as dpd_gt_{dpd}_{mon}m_flag"\
     .format(dpd=dpd, mon=mon) for mon in [1, 3, 6, 12, 24] for dpd in [0, 30, 60, 90, 180]]

 
expr_3k3 += [f"greatest("+", ".join([f"dpd_bucket_"+str(i) for i in range(1,mon+1)])+f") as max_dpd_{mon}m_flag" for mon in [3, 6, 12, 24]]
        
expr_3k3 += ["dpd_bucket_1 as max_dpd_1m_flag"]

In [7]:
agg_expr_3k1 = [] \
+ [F.expr(f"""{agg_func}(int(3mn_diff_retro_report_dt_flag = 1) * int(is_open = 1)* int({cat}_flag = 1) * int(row_num_first_{cat} = 1) * HIGHEST_CREDIT_OR_LOAN_AMOUNT) as First_loan_amt_{cat}""") for agg_func in ['max'] for cat in categories] \
+ [F.expr(f"""{agg_func}(int(3mn_diff_retro_report_dt_flag = 1) * int(is_open = 1)* int({cat}_flag = 1) * int(row_num_latest_{cat} = 1) * HIGHEST_CREDIT_OR_LOAN_AMOUNT * 1.0) as latest_loan_amt_{cat}""") for agg_func in ['max'] for cat in categories] \
+ [F.expr(f"""{agg_func}(HIGHEST_CREDIT_OR_LOAN_AMOUNT*{cat}_flag * is_open * 3mn_diff_retro_report_dt_flag) as avg_samt_{cat}""") for agg_func in ['mean'] for cat in categories] \
+ [F.expr(f"""{agg_func}(HIGHEST_CREDIT_OR_LOAN_AMOUNT*{cat}_flag * is_open * 3mn_diff_retro_report_dt_flag * 1.0) as {agg_func}_samt_{cat}""") for agg_func in ['max', 'min'] for cat in categories]



agg_expr_3k2 = []\
+ [F.expr(f"""sum(nbr_of_open_accnts_{cat}_{mn}M) as nbr_of_open_accnts_{cat}_{mn}M""") for cat in categories for mn in [3,6,12,24]] \
+ [F.expr(f"""sum(nbr_of_live_accnts_rep{cat}_{mn}M) as nbr_of_live_accnts_rep{cat}_{mn}M""") for cat in categories for mn in [3,6,12,24]]\
+ [F.expr(f"""sum(tot_live_bal_{cat}_{mn}M) as tot_live_bal_{cat}_{mn}M""") for cat in categories for mn in [3,6,12,24]]\
+ [F.expr(f"""sum(tot_live_sanction_{cat}_{mn}M) as tot_live_sanction_{cat}_{mn}M""") for cat in categories for mn in [3,6,12,24]] \
+ [F.expr(f"""{agg_func}(mon_util * {cat}_flag * {mon}mn_diff_retro_report_dt_flag * is_open) as avg_util_{cat}_{mon}M""") for agg_func in ['mean'] for cat in categories for mon in [3, 6, 12, 24]] \
+ [F.expr(f"""{agg_func}(month_diff_open_close * {cat}_flag * {mon}mn_diff_retro_report_dt_flag) as mons_diff_O_C_{agg_func}{cat}_{mon}M""") for agg_func in ['min', 'max', 'mean', 'median'] for cat in categories for mon in [3,6,12,24] ] \
+ [F.expr("mean(dateopened_diff) as avg_time_acct_open")] \
+ [F.expr("mean(lt10k_flag * dateopened_diff_lt10k) as avg_time_acct_open_lt10k")] \
+ [F.expr("mean(gt30k_flag * dateopened_diff_gt30k) as avg_time_acct_open_gt30k")]



agg_expr_3k3 = []\
+ [F.expr(f"""{agg_func}(balance_3k * {cat}_flag * dpd_gt_{dpd}_{mn}m_flag * dpd_present_{mn}M_flag) as del_bal_sum_{cat}_{mn}M_{dpd}_plus""") for agg_func in ['sum'] for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90','180']] \
+ [F.expr(f"""{agg_func}(HIGHEST_CREDIT_OR_LOAN_AMOUNT * month_lag_{mn}M * dpd_present_{mn}M_flag) as Total_sanctions_{mn}M""") for agg_func in ['sum'] for mn in [1,3,6,12,24] ] \
+ [F.expr(f"""{agg_func}(month_lag_{mn}M * dpd_present_{mn}M_flag) as Total_accounts_{mn}M""") for agg_func in ['sum'] for mn in [1,3,6,12,24] ] \
+ [F.expr(f"""{agg_func}({cat}_flag * dpd_gt_{dpd}_{mn}m_count * dpd_present_{mn}M_flag ) as nbr_times_{dpd}plus_{cat}_{mn}M""") for agg_func in ['max'] for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90','180']] \
+ [F.expr(f"""{agg_func}({cat}_flag * dpd_gt_{dpd}_{mn}m_flag * dpd_present_{mn}M_flag) as nbr_{dpd}plus_products_{cat}_{mn}M""") for agg_func in ['sum'] for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90','180']] \
+ [F.expr(f"""{agg_func}({cat}_flag * dpd_gt_{dpd}_{mn}m_flag * dpd_present_{mn}M_flag) as del_accounts_{cat}_{mn}M_{dpd}_plus""") for agg_func in ['sum'] for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90','180']] \
+ [F.expr(f"""{agg_func}(HIGHEST_CREDIT_OR_LOAN_AMOUNT * {cat}_flag * dpd_gt_{dpd}_{mn}m_flag * dpd_present_{mn}M_flag) as del_sanctions_{cat}_{mn}M_{dpd}_plus""") for agg_func in ['sum'] for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90','180']] \
+ [F.expr(f"""{agg_func}({cat}_flag * max_dpd_{mn}m_flag * dpd_present_{mn}M_flag) as max_dpd_{cat}_{mn}M_temp""") for agg_func in ['max'] for cat in categories2 for mn in [1,3,6,12,24]] 



expr_3k4 = ["*"] \
+ [f"round(tot_live_bal_{cat}_{mon}M / tot_live_sanction_{cat}_{mon}M, 3) as sum_util_{cat}_{mon}M" for cat in categories for mon in [3, 6, 12, 24]] \
+ [f"First_loan_amt_{cat}/latest_loan_amt_{cat} as ratio_fl_{cat}" for cat in categories] \
+ [f"max_samt_{cat}/min_samt_{cat} as ratio_max_min_{cat}" for cat in categories] \
+ [f"max_samt_{cat}/avg_samt_{cat} as ratio_max_avg_{cat}" for cat in categories] \
+ [" case when max_dpd_{cat}_{mn}M_temp = 0 then 0  \
         when max_dpd_{cat}_{mn}M_temp = 1 then 1  \
         when max_dpd_{cat}_{mn}M_temp = 2 then 30  \
         when max_dpd_{cat}_{mn}M_temp = 3 then 60  \
         when max_dpd_{cat}_{mn}M_temp = 4 then 90  \
         when max_dpd_{cat}_{mn}M_temp = 5 then 120 \
         when max_dpd_{cat}_{mn}M_temp = 6 then 180  \
         when max_dpd_{cat}_{mn}M_temp = 7 then 180 \
         else null end as max_dpd_{cat}_{mn}M".format(cat = cat, mn = mn) for cat in categories2 for mn in [1,3,6,12,24]]



expr_3k5 = ["*"] \
+ [f"del_bal_sum_{cat}_{mn}M_{dpd}_plus / Total_sanctions_{mn}M as ratio_del_bal{cat}_{mn}M" for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['180']] \
+ [f"del_accounts_{cat}_{mn}M_{dpd}_plus / Total_accounts_{mn}M as ratio_del_act{cat}_{mn}M" for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['180']] \
+ [f"del_sanctions_{cat}_{mn}M_{dpd}_plus / Total_sanctions_{mn}M as ratio_del_sanc{cat}_{mn}M" for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['180']] \
+ [f"del_bal_sum_{cat}_{mn}M_{dpd}_plus / Total_sanctions_{mn}M as ratio_del_bal{cat}_{mn}M_{dpd}_plus" for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90']] \
+ [f"del_accounts_{cat}_{mn}M_{dpd}_plus / Total_accounts_{mn}M as ratio_del_act{cat}_{mn}M_{dpd}_plus" for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90']] \
+ [f"del_sanctions_{cat}_{mn}M_{dpd}_plus / Total_sanctions_{mn}M as ratio_del_sanc{cat}_{mn}M_{dpd}_plus" for cat in categories2 for mn in [1,3,6,12,24] for dpd in ['0','30','60','90']]



In [8]:
agg_expr = agg_expr_3k1 + agg_expr_3k2 + agg_expr_3k3


w_3k1 = Window.partitionBy("MemberReference").orderBy("dateopened")
w_3k2 = Window.partitionBy("MemberReference").orderBy("dateopened_lt10k")
w_3k3 = Window.partitionBy("MemberReference").orderBy("dateopened_gt30k")



In [9]:
df = spark.read.parquet(prep_data_path + "prep_tl_data.parquet")

In [10]:
df.columns

['EnquiryControlNumber',
 'MemberReference',
 'acct_uniq_id',
 'Loan_Status',
 'Loan_Type',
 'Loan_Classification',
 'SANCTIONAMOUNT',
 'HIGHCREDIT',
 'BALANCE',
 'actual_emi',
 'CREDITLIMIT',
 'CASHLIMIT',
 'lastpayment',
 'Interest_Rates',
 'repaymenttenure',
 'Income',
 'monthly_annual_indicator',
 'net_gross_indicator',
 'occupation_code',
 'collateral_type',
 'collateral_value',
 'DATE_OPENED',
 'DATE_CLOSED',
 'DateReported_trades',
 'Pay_Hist_Start_Date',
 'Pay_Hist_END_Date',
 'Ownership_Type',
 'AMOUNT_OVERDUE',
 'Tenor_Frequency',
 'is_suit_filed_or_wilful_default',
 'is_written_off_or_settled',
 'written_off_amount',
 'written_off_amount_principal',
 'settlement_amt',
 'Dpd_string',
 'PL_Propensity_Score',
 'NTC_Score',
 'TU_Score',
 'ExclusionCode1',
 'ExclusionCode2',
 'ExclusionCode3',
 'ExclusionCode4',
 'ExclusionCode5',
 'ExclusionCode6',
 'ExclusionCode7',
 'ReasonCode1',
 'ReasonCode2',
 'ReasonCode3',
 'ReasonCode4',
 'ReasonCode5',
 'ErrorCode',
 'account_holder_ty

In [11]:
df1 = df\
.withColumn("balance_3k", F.when(F.col("balance") < 0,F.lit(0)).otherwise(F.col("balance")))\
.withColumn('mn_diff_retro_report', F.months_between(F.trunc('run_dt','month'), F.trunc('datereported', 'month')))\
.withColumn('HIGHEST_CREDIT_OR_LOAN_AMOUNT' , F.greatest(F.col('highcredit') , F.col('sanctionamount'), F.lit(0)))



In [12]:
df1.groupBy("ACCOUNT_TYPE").count().show(20,False)

+---------------------------------------------+-----+
|ACCOUNT_TYPE                                 |count|
+---------------------------------------------+-----+
|Loan to Professional                         |1    |
|Credit Card                                  |28   |
|Loan Against Bank Deposits                   |7    |
|Construction Equipment Loan                  |1    |
|Consumer Loan                                |12   |
|Housing Loan                                 |18   |
|Personal Loan                                |41   |
|Auto Loan (Personal)                         |11   |
|Business Loan - Priority Sector - Agriculture|20   |
|Gold Loan                                    |61   |
|Other                                        |6    |
|Property Loan                                |5    |
|Two-wheeler Loan                             |13   |
|Commercial Vehicle Loan                      |3    |
|Used Car Loan                                |1    |
+---------------------------

In [13]:
df2 = df1\
.withColumn("dateopened_lag", F.lag("dateopened").over(w_3k1))\
.withColumn("dateopened_diff", F.datediff("dateopened","dateopened_lag"))\
.withColumn("lt10k_flag", F.expr("case when HIGHEST_CREDIT_OR_LOAN_AMOUNT <= 10000 then 1 else null end"))\
.withColumn("gt30k_flag", F.expr("case when HIGHEST_CREDIT_OR_LOAN_AMOUNT > 30000 then 1 else null end"))\
.withColumn("dateopened_lt10k", F.expr("case when lt10k_flag = 1 then dateopened else null end"))\
.withColumn("dateopened_gt30k", F.expr("case when gt30k_flag = 1 then dateopened else null end"))\
.withColumn("dateopened_lag_lt10k", F.lag("dateopened_lt10k").over(w_3k2))\
.withColumn("dateopened_diff_lt10k", F.datediff("dateopened","dateopened_lag_lt10k"))\
.withColumn("dateopened_lag_gt30k", F.lag("dateopened_gt30k").over(w_3k3))\
.withColumn("dateopened_diff_gt30k", F.datediff("dateopened","dateopened_lag_gt30k"))


trd3k = df2\
.selectExpr(expr_3k1)\
.selectExpr(expr_3k2)\
.selectExpr(expr_3k3)\
.groupBy("MemberReference")\
.agg(*agg_expr)\
.replace(
    to_replace = 0 , 
    value = 0.1 ,
    subset = [f"min_samt_{cat}" for cat in categories] + [f"latest_loan_amt_{cat}" for cat in categories]
)\
.selectExpr(expr_3k4)\
.selectExpr(expr_3k5)\
.drop(*[f"max_dpd_{cat}_{mn}M_temp" for cat in categories2 for mn in [1,3,6,12,24] ])



In [14]:
len(trd3k.columns)

3098

In [15]:
# trd3k.show(1)

In [18]:
trd3k.write.mode("overwrite").parquet(prep_data_path + "cir_3k_features_spark.parquet")


25/05/13 13:11:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/13 13:11:33 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB


In [ ]:
quit()